In [508]:
import json
import base64
import qrcode
import requests
import datetime
import urllib.parse
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from didcomm.message import Attachment, AttachmentDataJson
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [509]:
import sys
import os
import time
import jpype
import jpype.imports
import json
try:
    sdk_gradle_home = '/Users/alex/Projects/rootid/didcomm-mediator/prism-cli-v1.4.1/lib'
except KeyError:
    print("ERROR: `ATALA_PRISM_JARS` variable is not set.")
    print("Please, set it to the directory with Atala PRISM SDK dependencies JARs.")
    sys.exit(1)
try:
    jpype.imports.registerDomain('sdk', alias='io')
except ImportError as err:
    print(err.msg)
    sys.exit(ERROR)
jpype.startJVM(
    classpath=[
        os.path.join(sdk_gradle_home, f) for f in os.listdir(sdk_gradle_home)
            if f.endswith('.jar')
    ]
)

OSError: JVM is already started

In [510]:
from sdk.iohk.atala.prism.protos import *
from sdk.iohk.atala.prism.api.node import *
from sdk.iohk.atala.prism.api.models import *
from sdk.iohk.atala.prism.api import *
from sdk.iohk.atala.prism.crypto.derivation import *
from sdk.iohk.atala.prism.crypto.keys import *
from sdk.iohk.atala.prism.identity import *
from kotlinx.serialization.json import *
from kotlinx.serialization import DeserializationStrategy
from sdk.iohk.atala.prism.crypto import MerkleInclusionProof



KeyGenerator = KeyGenerator.INSTANCE
KeyDerivation = KeyDerivation.INSTANCE
MasterKeyUsage = MasterKeyUsage.INSTANCE
IssuingKeyUsage = IssuingKeyUsage.INSTANCE
RevocationKeyUsage = RevocationKeyUsage.INSTANCE
AuthenticationKeyUsage = AuthenticationKeyUsage.INSTANCE
AtalaOperationStatus = AtalaOperationStatus.INSTANCE
PrismDID = PrismDid


In [528]:
def wait_until_confirmed(node_api: NodePublicApi, operation_id: AtalaOperationId):
    """Waits until operation is confirmed by Cardano network
    
    Confirmation doesn't necessarily mean that operation was applied.
    For example, it could be rejected because of an incorrect signature or other reasons.

    :param node_api: Atala PRISM Node API object
    :type node_api: NodePublicApi
    :param operation_id: Atala PRISM operation ID
    :type operation_id: AtalaOperationId
    """
    operation_status = int(node_api.getOperationInfo(operation_id).join().getStatus())
    while operation_status != AtalaOperationStatus.getCONFIRMED_AND_APPLIED() \
        and operation_status != AtalaOperationStatus.getCONFIRMED_AND_REJECTED():
        print(f"Current operation status: {AtalaOperationStatus.asString(operation_status)}")
        time.sleep(1)
        operation_status = int(node_api.getOperationInfo(operation_id).join().getStatus())

def dictToJsonObjt(json: dict):
    resp = {}
    for key in json:
        if isinstance(json[key], str) or isinstance(json[key], int) or isinstance(json[key], float): 
            resp[key] = JsonLiteral(json[key], True)
        elif isinstance(json[key], dict): 
            resp[key] = dictToJsonObjt(json[key])
        elif isinstance(json[key], list): 
            resp[key] = listToJsonArray(json[key])
    return JsonObject(resp)

def listToJsonArray(array: list):
    resp = jpype.java.util.ArrayList()
    for el in array:
        if isinstance(el, str) or isinstance(el, int) or isinstance(el, float): resp.add(JsonLiteral(el, True))
        elif isinstance(el, dict): 
            resp.add(dictToJsonObjt(el))
        elif isinstance(el, dict): 
            resp.add(listToJsonArray(el))
    return JsonArray(resp)


## Helper function

In [512]:
secrets_resolver = SecretsResolverDemo()

In [513]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [514]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


In [1]:
os.environ["ATALA_PRISM_JARS"]

KeyError: 'ATALA_PRISM_JARS'

In [515]:
# %load /Users/alex/Projects/rootid/didcomm-mediator/db_utils.py
''' DB Utilities '''
from pymongo import MongoClient
from bson.objectid import ObjectId
import datetime
import os
import urllib.parse


mongo = MongoClient(
    os.environ["DB_URL"],
    username=urllib.parse.quote_plus(os.environ["MONGODB_USER"]) if "MONGODB_USER" in os.environ else None,
    password=urllib.parse.quote_plus(os.environ["MONGODB_PASSWORD"]) if "MONGODB_PASSWORD" in os.environ else None,
    authSource="admin"
)


db = mongo.mediator

def get_connection(remote_did):
    ''' Get existing connection '''
    return db.connections.find_one({"remote_did": remote_did})

def create_connection(remote_did, local_did):
    ''' Create connection'''
    db.connections.insert_one({
        "remote_did": remote_did,
        "local_did": local_did,
        "creation_time": int(datetime.datetime.now().timestamp())
    })

def update_connection(remote_old_did, remote_new_did):
    ''' Update connection '''
    #TODO history of rotations
    db.connections.update_one(
        {"remote_did": remote_old_did}, {
            "$set": {"remote_did": remote_new_did, "update_time": int(datetime.datetime.now().timestamp())}
        }
    )

def get_oob_did():
    return db.oobs.find_one(sort=[('date', -1)])

def store_oob_did(did):
    db.oobs.insert_one(did)

def get_issuer_did():
    issuer_did = db.issuers.find_one(sort=[('date', -1)])
    return issuer_did if issuer_did is not None else None

def store_issuer_did(did):
    db.issuers.insert_one(did)


def store_vc(vc):
    db.vcs.insert_one(vc)

def get_vc(vc_id):
    return db.vcs.find_one({"id": vc_id})

def get_prism_holder_did():
    holder_did = db.prism_holder.find_one(sort=[('date', -1)])
    return holder_did if holder_did is not None else None

def store_prism_holder_did(did):
    db.prism_holder.insert_one(did)

def get_prism_did(did):
    return db.prism.find_one({"did": did})

def store_prism_did(did,seed):
    db.prism.insert_one({
        "did": did,
        "seed": seed
    })

def add_mediation(remote_did, routing_did, endpoint):
    ''' Add mediation info to connection '''
    db.connections.update_one(
        {"remote_did": remote_did}, {
            "$set": {
                        "isMediation": True, 
                        "routing_did": routing_did,
                        "endpoint": endpoint
                    }
        }
    )

def update_keys(remote_did, updates):
    ''' Add mediation keys '''
    connection = get_connection(remote_did)
    current_keys = connection["keylist"] if "keylist" in connection else []
    updated = []
    for update in updates:
        if update["action"] == "add":
            if update["recipient_did"] in current_keys:
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "add",
                        "result": "no_change"
                    })
            else:
                current_keys.append(update["recipient_did"])
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "add",
                        "result": "success"
                    })
        elif update["action"] == "remove":
            if update["recipient_did"] in current_keys:
                current_keys.remove(update["recipient_did"])
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "remove",
                        "result": "success"
                    })
            else:
                updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": "remove",
                        "result": "no_change"
                    })
        else:
            updated.append(
                    {
                        "recipient_did": update["recipient_did"],
                        "action": update["action"],
                        "result": "client_error"
                    })
        db.connections.update_one(
        {"remote_did": remote_did}, {
            "$set": {
                        "keylist": current_keys, 
                    }
        }
    )
    return updated

def get_message_status(remote_did, recipient_key):
    if recipient_key:
        count = db.messages.count_documents({"recipient_key": recipient_key},{"attachment":1})
    else:
        recipient_keys = db.connections.find_one({"remote_did":remote_did})["keylist"]
        count = db.messages.count_documents({"recipient_key": {"$in":recipient_keys}})
    return count

def get_messages(remote_did, recipient_key,limit):
    if recipient_key:
        return db.messages.find({"recipient_key": recipient_key},{"attachment":1}).limit(limit)
    else:
        recipient_keys = db.connections.find_one({"remote_did":remote_did})["keylist"]
        return list(db.messages.find({"recipient_key": {"$in":recipient_keys}}).limit(limit))

def add_message(recipient_key, attachment):
    # TODO verify that recipient_key belong to a registered peer
    db.messages.insert_one(
            {
                "recipient_key": recipient_key,
                "attachment": attachment,
                "datetime": int(datetime.datetime.now().timestamp())
            }
        )

def remove_messages(remote_did, message_id_list):
    #TODO verify that recipient_key belongs to remote_did
    for id in message_id_list:
        db.messages.delete_one({"_id": ObjectId(id)})
    return get_message_status(remote_did, None)

def get_short_url(oobid):
    short_url = db.shortUrls.find_one({"oobid": oobid},sort=[('date', -1)])
    return short_url

def store_short_url(short_url):
    db.shortUrls.insert_one(short_url)

def expire_short_url(oobid):
        # TODO order by date
        db.shortUrls.update_one(
            {"oobid": oobid}, {
                "$set": {
                            "expires_time": int(0)
                        }
            }
        )
    

### Reading issuer OOB message

In [516]:
# oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text
oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
print(oob_url)

http://127.0.0.1:8000?_oob=eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiNDY4M2JiOWQtMTRlOC00Y2Y5LWJmMjgtZmJkYmMyYjFlYjBiIiwiZnJvbSI6ImRpZDpwZWVyOjIuRXo2TFNnU2kxVm1rS01UY1dLSk1meHpKZ1ExUVFZbVZWVVhXeHlqQVN1c0Q2dTZQRC5WejZNa2d4dG5BRWpmc2FYcnl1SFNWWHRvN2t3V3NuRWs1TnAyU1hkVVdvZ1A3OWRBLlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCIsImJvZHkiOnsiZ29hbF9jb2RlIjoicmVxdWVzdC1tZWRpYXRlIiwiZ29hbCI6IlJlcXVlc3RNZWRpYXRlIiwiYWNjZXB0IjpbImRpZGNvbW0vdjIiLCJkaWRjb21tL2FpcDI7ZW52PXJmYzU4NyJdfX0


In [517]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
print(received_msg_decoded)

{'type': 'https://didcomm.org/out-of-band/2.0/invitation', 'id': '4683bb9d-14e8-4cf9-bf28-fbdbc2b1eb0b', 'from': 'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0', 'body': {'goal_code': 'request-mediate', 'goal': 'RequestMediate', 'accept': ['didcomm/v2', 'didcomm/aip2;env=rfc587']}}


In [518]:
#Mediator DID
received_msg_decoded['from']

'did:peer:2.Ez6LSgSi1VmkKMTcWKJMfxzJgQ1QQYmVVUXWxyjASusD6u6PD.Vz6MkgxtnAEjfsaXryuHSVXto7kwWsnEk5Np2SXdUWogP79dA.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXX0'

In [519]:

holder_did = await create_peer_did(1,1, service_endpoint="https://www.example.com/holder")
print("Holder's DID:", holder_did)

Holder's DID: did:peer:2.Ez6LSqtFjAcpb8ovC7HiXFnxJLGMKjFnrE3ANpHtBSNL6xhJ8.Vz6Mko4zxEyErQLjfH9soCLJP2xxdBDGH2nDXBj9RHHndZEYU.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwczovL3d3dy5leGFtcGxlLmNvbS9ob2xkZXIiLCJhIjpbImRpZGNvbW0vdjIiXX0


Also, the holder needs to provide a did:prism where the credential will be issued to.

In [520]:
get_prism_holder_did()

{'_id': ObjectId('632636218817eb4fe45b44a8'),
 'did': 'did:prism:8ade34754487af867b813d008d3704451a0c1df88090c1947d6f901857fd5ada:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQJ-gKvROnRTlmvLtBGxikIg07oaJZc2B5bU4MrAGtwDkhI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhA6I4nH3zH1vn1Azo_WLGr4Btz5eb9cHA-KdQq5lIsGUoEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiECJiZwIzuQQen4LL-OPOBUCs9zsrD0u2Z_q2PhZzx0Nrc',
 'master0': 'd8fd61eb731a043302df05cc2932f306777ea227481685f9227f493382dea319',
 'issuing0': '3f772360c0a83dab51d69a50a21f67efa6f8b8ebccb5ab4e172ec82873b1b9a3',
 'revocation0': 'aa4ad746e1aa6def9aeebbc917c0a765440267929362540c9d984953a0de4adc',
 'date': 1663448609000,
 'seed': b'q\x89\xb5\x97\x00\x81\x1c!)[\xcd\xa6zS\x00\xd6\xf3\x0c\xfc \x07\x12}\xb9\xc4\xc7d\x92H\x7f\xec\xd8:\x16]\xca\xc2\xcf2U\xfb\x06\xd1\xf2\xf8/\xe6\xddtcO\xa3.9\xadB\xe4\x0b\xa1\x02i\x17\xdf*'}

In [584]:
#print code text for foucntion verify_prism_credential
import inspect
print(inspect.getsource(verify_prism_credential))

def verify_prism_credential(credential):
    holder_signed_credential = credential['proof']['proofValue']
    hsc = JsonBasedCredential.fromString(holder_signed_credential)
    inpo = {"hash":credential['proof']['proofHash'],"index":0,"siblings":[]}
    pff = MerkleInclusionProof.decode(dictToJsonObjt(inpo).toString())
    environment = "ppp.atalaprism.io"
    node_auth_api = NodeAuthApiAsyncImpl(GrpcOptions("http", environment, 50053))
    # Verifier, who owns credentialClam, can easily verify the validity of the credentials
    credential_verification_result = node_auth_api.verify(
        hsc,
        pff
    ).join()

    verification_errors = credential_verification_result.getVerificationErrors()
    return verification_errors



In [521]:
#get PRISM DIDS
print(get_prism_holder_did()['did'])
print(get_issuer_did()['did'])

did:prism:8ade34754487af867b813d008d3704451a0c1df88090c1947d6f901857fd5ada:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQJ-gKvROnRTlmvLtBGxikIg07oaJZc2B5bU4MrAGtwDkhI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhA6I4nH3zH1vn1Azo_WLGr4Btz5eb9cHA-KdQq5lIsGUoEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiECJiZwIzuQQen4LL-OPOBUCs9zsrD0u2Z_q2PhZzx0Nrc
did:prism:fb727d449bd219021503d3f8c63a221cc1861a90e93b5c017b139aafdd2c14d9:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQNJuG3G_ZCEg9YHAXQpvLagXQTb8emtVTjBh_m66F5uoRI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhAiRByhWe4k1MTd7cbmRxrthrV7WPsRaKccZeDIAWoGJlEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiEDTkb_zwgk-ILQcH3tW3kCAiuuGcRcjiDlQ3h8eS0MmPk


The following is a JSON-LD credential request that the holder will submmit to the issuer.

In [522]:
credential_request = {
    "credential": {
        "@context": 
        [
            "https://www.w3.org/2018/credentials/v1",
            "https://www.w3.org/2018/credentials/examples/v1"
        ],
        "id": str(uuid.uuid4()),
        "type": ["VerifiableCredential", "UniversityDegreeCredential"],
        "issuer": get_issuer_did()['did'],
        "issuanceDate": datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%SZ"),
        "credentialSubject": 
            {
              "id": get_prism_holder_did()['did'],
              "degree": 
                {
                    "type": "BachelorDegree",
                    "name": "Bachelor of Science and Arts"
                  }
            },
        "options": {
            "proofType": "EcdsaSecp256k1Signature2019"
        }
    }
}
#assert issuer contains "did:prism"
assert "did:prism" in credential_request['credential']['issuer']
#assert credentialSubject id  contains "did:prism"
assert "did:prism" in credential_request['credential']['credentialSubject']['id']

Finally the holder prepares the didcomm message following issue-credential/3.0 protocol:

In [523]:
holder_response_message = Message(
    custom_headers = [{
        "return_route": "all"}],
    id = str(uuid.uuid4()),
    #pthid = received_msg_decoded["id"],
    type = "https://didcomm.org/issue-credential/3.0/request-credential",
    frm = holder_did,
    to = [received_msg_decoded["from"]],
    body = {
        "goal_code": "issue-credential",
        "comment": "some comment"
    },
    attachments = [
        Attachment(
                id=str(uuid.uuid4()),
                media_type= "application/json",
                format= "aries/ld-proof-vc-detail@v1.0",
                data=AttachmentDataJson(json=credential_request)
                )
    ]
                        
)

holder_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = holder_response_message,
    frm = holder_did,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending the message to Mediator

From the issuer DID obtained in the OOB QR Code, we can get the DID document:

In [524]:
issuer_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
issuer_endpoint = issuer_did_doc["service"][0]["serviceEndpoint"]
print(issuer_endpoint)

http://127.0.0.1:8000


And from there get the issuer's endpoint

In [525]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(issuer_endpoint, headers=headers, data = holder_packed_msg.packed_msg)
resp

<Response [500]>

Finally sending the request to the issuer:

# Wait

In [271]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)

### Issued Verifiable Credential Received

In [272]:
credential_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
credential =credential_unpack_msg.message.attachments[0].data.json

In [526]:
credential

{'@context': ['https://www.w3.org/2018/credentials/v1',
  'https://www.w3.org/2018/credentials/examples/v1'],
 'id': '15d3604e-919c-4c2b-a134-40c63dd75eca',
 'type': ['VerifiableCredential', 'UniversityDegreeCredential'],
 'issuer': 'did:prism:fb727d449bd219021503d3f8c63a221cc1861a90e93b5c017b139aafdd2c14d9:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQNJuG3G_ZCEg9YHAXQpvLagXQTb8emtVTjBh_m66F5uoRI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhAiRByhWe4k1MTd7cbmRxrthrV7WPsRaKccZeDIAWoGJlEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiEDTkb_zwgk-ILQcH3tW3kCAiuuGcRcjiDlQ3h8eS0MmPk',
 'issuanceDate': '2022-09-17T16:21:22Z',
 'credentialSubject': {'id': 'did:prism:8ade34754487af867b813d008d3704451a0c1df88090c1947d6f901857fd5ada:Cr8BCrwBEjsKB21hc3RlcjAQAUouCglzZWNwMjU2azESIQJ-gKvROnRTlmvLtBGxikIg07oaJZc2B5bU4MrAGtwDkhI8Cghpc3N1aW5nMBACSi4KCXNlY3AyNTZrMRIhA6I4nH3zH1vn1Azo_WLGr4Btz5eb9cHA-KdQq5lIsGUoEj8KC3Jldm9jYXRpb24wEAVKLgoJc2VjcDI1NmsxEiECJiZwIzuQQen4LL-OPOBUCs9zsrD0u2Z_q2PhZzx0Nrc',
  'degree': {'type': 

In [581]:
verify_prism_credential(credential)

Sep 18, 2022 4:39:19 PM io.grpc.internal.ManagedChannelOrphanWrapper$ManagedChannelReference cleanQueue
SEVERE: *~*~*~ Channel ManagedChannelImpl{logId=132, target=ppp.atalaprism.io:50053} was not shutdown properly!!! ~*~*~*
    Make sure to call shutdown()/shutdownNow() and wait until awaitTermination() returns true.
Sep 18, 2022 4:39:19 PM io.grpc.internal.ManagedChannelOrphanWrapper$ManagedChannelReference cleanQueue
SEVERE: *~*~*~ Channel ManagedChannelImpl{logId=134, target=ppp.atalaprism.io:50053} was not shutdown properly!!! ~*~*~*
    Make sure to call shutdown()/shutdownNow() and wait until awaitTermination() returns true.
Sep 18, 2022 4:39:19 PM io.grpc.internal.ManagedChannelOrphanWrapper$ManagedChannelReference cleanQueue
SEVERE: *~*~*~ Channel ManagedChannelImpl{logId=126, target=ppp.atalaprism.io:50053} was not shutdown properly!!! ~*~*~*
    Make sure to call shutdown()/shutdownNow() and wait until awaitTermination() returns true.
Sep 18, 2022 4:39:19 PM io.grpc.internal

<java object 'kotlin.collections.EmptyList'>

In [ ]:
#generate oob for vp request from the mediator

In [275]:
#Notebook sends a propose-presentation message to the mediator getting the thid from the oob url 

In [ ]:
#Mediator sends a request-presentation message to the holder with a presentation definition

In [ ]:
#Notebook presents proof for both the credential and the submission

In [ ]:
#mediatior send an ack to the notebook

In [546]:
presentation_definition = {'presentation_definition': 
{'presentation_definition': {'id': '32f54163-7166-48f1-93d8-ff217bdb0654',
   'input_descriptors': [{'id': '66685f64-7166-5717-b3fc-ff217bdb9999',
     'constraints': {'fields': [{'path': ['$.type'],
        'filter': {'type': 'string',
         'pattern': 'UniversityDegreeCredential'}}]}}]}},
 'options': {'challenge': '3fa85f64-5717-4562-b3fc-2c963f66afa7',
  'domain': '4jt78h47fh47'}}

In [550]:
presentation_definition['options']

{'challenge': '3fa85f64-5717-4562-b3fc-2c963f66afa7', 'domain': '4jt78h47fh47'}

In [555]:
import hashlib, jsonld
proof_hash = hashlib.sha256(presentation_definition['options']['challenge'].encode('utf-8'))
normalized_proof = jsonld.normalize(credential, {'algorithm': 'URDNA2015', 'format': 'application/n-quads'})
proof_hash.update(normalized_proof.encode('utf-8'))
proof_hash.hexdigest()

In [564]:
proof_hash.digest()

b'\x8f\xb6\x01\x84\xd6-\xe5\x89\x18\xae\x08\x14\xc9\xa6\xbfp\xa4\xdf\xf0\xf6\xac\xa441\xef\x88\x17\x1d\x81\x84\xcd%'

In [577]:
import ecdsa
from hashlib import sha256
hex_private_holder = get_prism_holder_did()['master0']
sk = ecdsa.SigningKey.from_string(bytes.fromhex(hex_private_holder), curve=ecdsa.SECP256k1, hashfunc=sha256) # the default is sha1
sig = sk.sign(bytes.fromhex(proof_hash.hexdigest()))
sig.hex()

'dabd762472fbec4a455da5157659f74a3c32abb2ea88048d0d153b1c468be7c3e0445960eee39585420a4e82e48e2d31df42f599a6c8197bfad9f4d01d94d878'

In [579]:
import ecdsa
def verify_proofhash(proof_hash_hexdigest, signature_hex, public_key):
    seed_holder = get_prism_holder_did()['seed']
    holder_keys = prepare_keys_from_seed(get_prism_holder_did()['seed'])
    public_key_hex_holder = holder_keys['master0'].getPublicKey().getHexEncoded()
    public_key_hex_holder
    vk = ecdsa.VerifyingKey.from_string(bytes.fromhex(public_key_hex_holder), curve=ecdsa.SECP256k1, hashfunc=sha256) # the default is sha1
    try:
        return vk.verify(bytes.fromhex(signature_hex), bytes.fromhex(proof_hash_hexdigest()))
    except e as Exception:
        return False
df39352dbf905cadc9a495400706c53acfb90c81ec403aceeb2ed9757ab66f07b96d8d436e1fa23aeb9b79a10b623ed067f6451f8ddab9662192bd6294325186

NameError: name 'prepare_keys_from_seed' is not defined

In [578]:

vk = ecdsa.VerifyingKey.from_string(bytes.fromhex('047e80abd13a7453966bcbb411b18a4220d3ba1a2597360796d4e0cac01adc03924a3f5c9eff729ddf9f155dc62e5afa1ddb1f6db27b31f98558c03c8fab545062'), curve=ecdsa.SECP256k1, hashfunc=sha256) # the default is sha1
vk.verify(bytes.fromhex(sig.hex()), bytes.fromhex(proof_hash.hexdigest()))



True

In [568]:
sk = ecdsa.VerifyingKey.from_string(bytes.fromhex('04ed29c5d8120c1ed0111b10d6c83a91b2aaafb4d8a81de739379d618b705448e0cda1ba323feb6a19639fb90ca59b9e8fb55ac72e7f3bade3c0774c2f37232e4c'), curve=ecdsa.SECP256k1, hashfunc=sha256) # the default is sha1
vk.verify(bytes.fromhex('56d8789c4f3b36e3a9849f87ed46ef954aae1b5ee75da42912036bd975e05462ec3bf393cd57edb90b95bfc9b1d854a95dcd37cefea9512897f5994b11153a0f'), b"domain")


True

In [455]:
sk = ecdsa.SigningKey.from_string(bytes.fromhex('b7174a25a5b2077ae40f89d3ee0584ddbc2546a1ded5e9991da1f83f1a09bb97'), curve=ecdsa.SECP256k1, hashfunc=sha256) # the default is sha1


NameError: name 'ecdsa' is not defined

In [541]:
obj = {
  "type": "https://didcomm.org/out-of-band/2.0/invitation",
  "id": "599f3638-b563-4937-9487-dfe55099d900",
  "from": "did:example:verifier",
  "body": {
      "goal_code": "streamlined-vp",
      "accept": ["didcomm/v2"]
  }
}
#To encode this message, remove all json whitespace and Base 64 URL encode. The result should look like this, for the example above:
encoded_msg = base64.urlsafe_b64encode(json.dumps(obj).replace(' ', '').encode('utf-8')).decode('utf-8')
print(encoded_msg)

eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiNTk5ZjM2MzgtYjU2My00OTM3LTk0ODctZGZlNTUwOTlkOTAwIiwiZnJvbSI6ImRpZDpleGFtcGxlOnZlcmlmaWVyIiwiYm9keSI6eyJnb2FsX2NvZGUiOiJzdHJlYW1saW5lZC12cCIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ==


In [543]:
encoded_msg=='eyJ0eXBlIjoiaHR0cHM6Ly9kaWRjb21tLm9yZy9vdXQtb2YtYmFuZC8yLjAvaW52aXRhdGlvbiIsImlkIjoiNTk5ZjM2MzgtYjU2My00OTM3LTk0ODctZGZlNTUwOTlkOTAwIiwiZnJvbSI6ImRpZDpleGFtcGxlOnZlcmlmaWVyIiwiYm9keSI6eyJnb2FsX2NvZGUiOiJzdHJlYW1saW5lZC12cCIsImFjY2VwdCI6WyJkaWRjb21tL3YyIl19fQ'

False

In [538]:
json.dumps(obj).replace(' ', '')

'{"type":"https://didcomm.org/out-of-band/2.0/invitation","id":"599f3638-b563-4937-9487-dfe55099d900","from":"did:example:verifier","body":{"goal_code":"streamlined-vp","accept":["didcomm/v2"]}}'